In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
from scipy.optimize import minimize
from sklearn.externals import joblib

from tqdm import tqdm
import multiprocessing as mp
from joblib import Parallel, delayed

import warnings
warnings.filterwarnings('ignore')

In [2]:
R_E = 6371000
depth_icecube = 1950
cut = 77.0 / 180 * np.pi


def calc_range(depth, zenith):
    b = R_E - depth_icecube + depth
    return -b * np.cos(zenith) + np.sqrt((R_E ** 2 - b ** 2) * np.sin(zenith) ** 2 + R_E ** 2 * np.cos(zenith) ** 2) + np.finfo(float).max * (R_E ** 2 - b ** 2 < 0.0)

r_gen = 800
h_gen = 1600
A_gen = 2 * np.pi * r_gen * h_gen + 2 * np.pi * r_gen**2

In [3]:
eMin = 400
eMax = 30000

data_dir = '/home/sninfa/jupyter/data/400-30k_10Bins'

area = joblib.load('%s/effArea_mgs_corsica_total.pickle' % data_dir)
input_df = joblib.load('%s/df_corsica_est.pickle' % data_dir)

input_df['range'] = calc_range(input_df.stop_z, input_df.zenith)
input_df['range_log'] = np.log10(input_df.range)
input_df['energy_log'] = np.log10(input_df.energy_stop)

input_df = input_df[(input_df.single_stopping > 0.79) & (input_df.quality < -0.6) & (input_df.zenith < cut) & (input_df.energy_stop > 0.0)]

input_df['weight_norm'] = input_df.weight.values / input_df.weight.sum()
input_df['weight_G4_norm'] = input_df.weight_G4.values / input_df.weight_G4.sum()
input_df['weight_H_norm'] = input_df.weight_H.values / input_df.weight_H.sum()

In [4]:
nbins_E = 10
nbins_r = 10

binning_E = np.logspace(np.log10(eMin), np.log10(eMax), nbins_E + 1)
binning_r = np.logspace(np.log10(2000), np.log10(13000), nbins_r + 1)

binning_idx_r = np.arange(nbins_r + 3) - 0.5
binning_idx_E = np.arange(nbins_E + 3) - 0.5

acceptance = area.values / A_gen
acceptance = np.insert(acceptance, 0 , 1.0)
acceptance = np.append(acceptance, 1.0)

In [5]:
cmap = plt.cm.magma
cmap.set_under('w')

In [6]:
def llh_poisson(A, f_est, g):
    return np.sum(A.dot(f_est) - g * np.log(np.abs(A.dot(f_est)) + 1e-8))
    # return np.sum(A.dot(f_est) - g * np.log(np.abs(A.dot(f_est / acceptance)) + 1e-8))

In [7]:
def only_positive(f_est):
    return np.finfo('float').max * (f_est < 0.0).any()

In [8]:
def C_matrix(n):
    I = np.eye(n)
    C = 2.0 * I - np.roll(I, 1) - np.roll(I, -1)
    return C

In [9]:
def tikhonov_reg(f_est, tau):
    C = C_matrix(len(f_est) - 2)
    return tau * np.sum(C.dot(np.log(np.abs(f_est[1:-1] / acceptance[1:-1]) + 10e+8)) ** 2)

In [10]:
def mcmc(x0, fun, step_size=1.5, n_steps=10000, print_acceptance=False, print_results=False):
    x = [x0]
    f = [fun(x0)]
    acc = 0
    for _ in range(n_steps):
        x_new = x[-1] + step_size * np.random.randn(len(x0))
        f_new = fun(x_new)
        prop_eval = -np.log(np.random.rand()) > f_new - f[-1]
        if prop_eval:
            x.append(x_new)
            f.append(f_new)
            acc += 1
        else:
            if print_results:
                print 'x_new: {}'.format(x_new)
                print 'f_new: {}'.format(f_new)
            x.append(x[-1])
            f.append(f[-1])
    if print_acceptance:
        print('{}% of proposed steps accepted.'.format(100 * acc / n_steps))
    return np.array(x), np.array(f)

In [11]:
def get_pull(pull_id, tau, output=None, max_iter=100000, model='G3'):   
    np.random.seed()
    permute = np.random.permutation(len(input_df))    
    
    # mc = input_df.loc[permute[input_df.index % 2 != 0]]
    # data = input_df.loc[permute[input_df.index % 2 == 0]]
    
    if model == 'G3':
        sel = np.random.choice(input_df.index, len(input_df.index)/2, p=input_df.loc[input_df.index, "weight_norm"], replace=False)
    elif model == 'G4':
        sel = np.random.choice(input_df.index, len(input_df.index)/2, p=input_df.loc[input_df.index, "weight_G4_norm"], replace=False)
    elif model == 'H':
        sel = np.random.choice(input_df.index, len(input_df.index)/2, p=input_df.loc[input_df.index, "weight_H_norm"], replace=False)
    else:
        sel = [-1]
        return 'Failed to select a model!!! (%s)' % model
        
    data = input_df[input_df.index.isin(sel)]    
    mc = input_df[~input_df.index.isin(sel)]

    mc['energy_idx'] = np.digitize(mc.energy_stop, binning_E)
    mc['range_idx'] = np.digitize(mc.range, binning_r)

    data['energy_idx'] = np.digitize(data.energy_stop, binning_E)
    data['range_idx'] = np.digitize(data.range, binning_r)

    H, _, _ = np.histogram2d(mc['range_idx'], mc['energy_idx'], (binning_idx_r, binning_idx_E), weights=mc['weight_norm'])
    A = H / np.sum(H, axis=0)
    A = np.nan_to_num(A)

    f_data, _ = np.histogram(data.energy_idx, binning_idx_E)
    g_data, _ = np.histogram(data.range_idx, binning_idx_r)

    function = lambda f_est: llh_poisson(A, f_est, g_data)\
                       + only_positive(f_est)\
                       + tikhonov_reg(f_est, tau)

    result = minimize(function, x0=f_data, method='Nelder-Mead', options={'maxiter': 10000})

    x0 = 50.0 * np.ones(len(f_data))

    x_sample, f_sample = mcmc(x0, function, step_size=1.2, n_steps=10000)
    x_sample, f_sample = mcmc(result.x, function, step_size=1.2, n_steps=100000, print_acceptance=False, print_results=False)
        
    if output is None:
        return [f_data, np.median(x_sample, axis=0), np.std(x_sample, axis=0)]
    else:
        output.put([f_data, np.median(x_sample, axis=0), np.std(x_sample, axis=0)])

In [12]:
def plot_pulls(pull):
    pull = np.swapaxes(pull, 0, 1)
    fig, ax = plt.subplots(11, 2, figsize=(7, 16), sharex=True)
    pull_bins = np.linspace(-3.0, 3.0, 20)
    t = np.linspace(-3.0, 3.0, 100)
    y_gauss = np.exp(-t ** 2 / 2.0) / np.sqrt(2.0 * np.pi)
    for i in range(nbins_E + 2):
        idx_1, idx_2 = np.unravel_index(i, ax.shape)
        p_stat = ((pull[0] - pull[1]) / pull[2])[:,i]
        p_stat = np.array(filter(lambda x : np.abs(x) < 10, p_stat))
        ax[idx_1, idx_2].hist(p_stat, pull_bins, normed=True)
        ax[idx_1, idx_2].plot(t, y_gauss)
        ax[idx_1, idx_2].set_xlim([-3.0, 3.0])
        ax[idx_1, idx_2].legend([], title="Bin %i\nMean %.2f\nStd: %.2f" % (i, np.mean(p_stat), np.std(p_stat)),
                                loc="upper right")
    ax[-1, 0].set_xlabel("Pull Statistic")
    ax[-1, 1].set_xlabel("Pull Statistic")
    # fig.delaxes(ax[-1,-1])
    plt.tight_layout()
    plt.show()

In [13]:
reg_coeff_list = [0.001,0.01,0.1,1,10,100]

In [14]:
pulls = []
for tau in reg_coeff_list:
    print '***Begin pulls for tau = %f' % tau
    pulls += [Parallel(n_jobs=8)(delayed(get_pull)(i, tau, model='G3') for i in range(1000))]

***Begin pulls for tau = 0.001000
***Begin pulls for tau = 0.010000
***Begin pulls for tau = 0.100000
***Begin pulls for tau = 1.000000
***Begin pulls for tau = 10.000000
***Begin pulls for tau = 100.000000


In [15]:
def plot_bias(pull_results):
    fig, ax = plt.subplots(len(reg_coeff_list)/2, 2, figsize=(16, 16), sharex=True, sharey=True)
    for i, tau in enumerate(reg_coeff_list):
        pull = pull_results[i]
        pull = np.swapaxes(pull, 0, 1)
        idx_1, idx_2 = np.unravel_index(i, ax.shape)
        data_list = []
        for j in range(nbins_E + 2):
            p_stat = ((pull[0] - pull[1]) / pull[2])[:,j]
            p_stat = np.array(filter(lambda x : np.abs(x) < 10, p_stat))
            data_list.append([j, np.mean(p_stat), np.std(p_stat)])
        data_list = np.array(data_list)
        ax[idx_1, idx_2].errorbar(x=data_list[:,0], y=data_list[:,1], yerr=data_list[:,2], fmt='x', label=r'$\tau = %i$' % tau, capsize=4, color='green')
        ax[idx_1, idx_2].hlines(y=[-1, 1], xmin=0, xmax=11, linestyles='--', color='red')
        ax[idx_1, idx_2].xlim = (-1, 22)
        ax[idx_1, idx_2].legend(loc="best")  
        if idx_2 == 0:
            ax[idx_1, idx_2].set_ylabel('pull statistic mean')
        if idx_1 == ax.shape[0] - 1:
            ax[idx_1, idx_2].set_xlabel('bin')
    plt.show()

In [16]:
joblib.dump(pulls, '/home/sninfa/jupyter/data/unfolding_v4_G3_output_weighted.pickle')

['/home/sninfa/jupyter/data/unfolding_v4_G3_output_weighted.pickle']